# 物件偵測

## AUTHENTICATE: 驗證用戶端

In [3]:
from azure.cognitiveservices.vision.computervision \
import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

SUBSCRIPTION_KEY = "94020429c5874d348734a8d2a7fd3b5d"
ENDPOINT = "https://computertest.cognitiveservices.azure.com/"
CV_CLIENT = ComputerVisionClient(
    ENDPOINT, CognitiveServicesCredentials(SUBSCRIPTION_KEY)
)

In [6]:
from io import BytesIO
import requests
from PIL import Image, ImageDraw, ImageFont

url = "https://i.imgur.com/Js5H6Qa.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [10]:
draw = ImageDraw.Draw(img)
font_size = int(5e-2 * img.size[1])
fnt = ImageFont.truetype(
  "./TaipeiSansTCBeta-Regular.ttf",
  size=font_size)

In [11]:
object_detection = CV_CLIENT.detect_objects(url)
if len(object_detection.objects) > 0:
    for obj in object_detection.objects:
        left = obj.rectangle.x
        top = obj.rectangle.y
        right = obj.rectangle.x + obj.rectangle.w
        bot = obj.rectangle.y + obj.rectangle.h
        name = obj.object_property
        confidence = obj.confidence
        print("{} at location {}, {}, {}, {}".format(
          name, left, right, top, bot))

chair at location 819, 1011, 7, 284
chair at location 479, 661, 70, 408
tabby cat at location 341, 1082, 411, 859


In [13]:
draw.rectangle(
          [left, top, right, bot],
          outline=(255, 0, 0), width=3)
draw.text(
            [left, top + font_size],
            "{0} {1:0.1f}".format(name, confidence * 100),
            fill=(255, 0, 0),
            font=fnt,
        )

In [14]:
img.save("output.png")

# OCR偵測

In [15]:
from io import BytesIO
import requests
from PIL import Image, ImageDraw, ImageFont

url = "https://i.imgur.com/qyWiqQv.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [16]:
draw = ImageDraw.Draw(img)
font_size = int(5e-2 * img.size[1])
fnt = ImageFont.truetype(
  "./TaipeiSansTCBeta-Regular.ttf",
  size=font_size)

In [17]:
ocr_results = CV_CLIENT.read(url, raw=True)
operation_location_remote = \
ocr_results.headers["Operation-Location"]
operation_id = operation_location_remote.split("/")[-1]

In [18]:
status = ["notStarted", "running"]
while True:
    get_handw_text_results = \
    CV_CLIENT.get_read_result(operation_id)
    if get_handw_text_results.status not in status:
        break
    time.sleep(1)

In [22]:
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
succeeded = OperationStatusCodes.succeeded

if get_handw_text_results.status == succeeded:
    res = get_handw_text_results.analyze_result.read_results
    for text_result in res:
        for line in text_result.lines:
            bounding_box = line.bounding_box
            bounding_box += bounding_box[:2]
            draw.line(
                line.bounding_box, 
                fill=(255, 0, 0), 
                width=int(font_size / 10)
            )
            left = line.bounding_box[0]
            top = line.bounding_box[1]
            draw.text(
                [left, top - font_size],
                line.text,
                fill=(0, 255, 255),
                font=fnt,
            )
            
img.save("output.png")            

### 產生描述

In [36]:
url = "https://cdn2.lamag.com/wp-content/uploads/sites/6/2015/02/puppy.jpg"
description_results = CV_CLIENT.describe_image(url)
output = ""
for caption in description_results.captions:
    output += "'{}' with confidence {:.2f}% \n".format(
        caption.text, caption.confidence * 100
    )
print(output)

'a dog running with a ball in its mouth' with confidence 60.56% 

